In [2]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd       
train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)
# train.shape should be (25000,3)


test = pd.read_csv("testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [4]:
import nltk

In [5]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:

import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

In [9]:

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
print(eng_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
def review_cleaner(review):
    '''
    Clean and preprocess a review.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    review = bs.BeautifulSoup(review).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
    
    #3. Remove punctuation
    review = re.sub("[^a-zA-Z]", " ",review)
    
    #4. Tokenize into words (all lower case)
    review = review.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    review = [w for w in review if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    review = ' '.join(review+emoticons)
    # add emoticons to the end

    return(review)

In [11]:
%%time

num_reviews = len(train['review'])

review_clean_original = []

for i in range(0,num_reviews):
    if( (i+1)%500 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    review_clean_original.append(review_cleaner(train['review'][i]))

Done with 500 reviews
Done with 1000 reviews
Done with 1500 reviews
Done with 2000 reviews
Done with 2500 reviews
Done with 3000 reviews
Done with 3500 reviews
Done with 4000 reviews
Done with 4500 reviews
Done with 5000 reviews
Done with 5500 reviews
Done with 6000 reviews
Done with 6500 reviews
Done with 7000 reviews
Done with 7500 reviews
Done with 8000 reviews
Done with 8500 reviews
Done with 9000 reviews
Done with 9500 reviews
Done with 10000 reviews
Done with 10500 reviews
Done with 11000 reviews
Done with 11500 reviews
Done with 12000 reviews
Done with 12500 reviews
Done with 13000 reviews
Done with 13500 reviews
Done with 14000 reviews
Done with 14500 reviews
Done with 15000 reviews
Done with 15500 reviews
Done with 16000 reviews
Done with 16500 reviews
Done with 17000 reviews
Done with 17500 reviews
Done with 18000 reviews
Done with 18500 reviews
Done with 19000 reviews
Done with 19500 reviews
Done with 20000 reviews
Done with 20500 reviews
Done with 21000 reviews
Done with 21

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
    review_clean_original, train['sentiment'], random_state=0, test_size=.2)


# CountVectorizer can actucally handle a lot of the preprocessing for us
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

In [13]:

%%time
# Transform the text data to feature
# Only fit training data (to mimic real world)

vectorizer.fit(X_train)


Wall time: 5.98 s


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=5000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [14]:
print(vectorizer.get_feature_names()[:10])

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely']


In [15]:

train_bag = vectorizer.transform(X_train) #transform to a feature matrix
test_bag = vectorizer.transform(X_test)

In [16]:

print(train_bag.toarray().shape) # 20,000 reviews, 2,000 feartures. just as expected
print(test_bag.toarray().shape)

(20000, 5000)
(5000, 5000)


In [17]:
print(type(train_bag))# sparse matrix representation

print(train_bag)


<class 'scipy.sparse.csr.csr_matrix'>
  (0, 37)	1
  (0, 41)	1
  (0, 46)	1
  (0, 51)	1
  (0, 58)	1
  (0, 103)	1
  (0, 126)	1
  (0, 142)	2
  (0, 145)	1
  (0, 147)	1
  (0, 162)	1
  (0, 194)	2
  (0, 205)	1
  (0, 265)	1
  (0, 286)	1
  (0, 315)	1
  (0, 327)	2
  (0, 335)	1
  (0, 368)	1
  (0, 395)	1
  (0, 411)	1
  (0, 436)	1
  (0, 475)	1
  (0, 480)	1
  (0, 485)	1
  :	:
  (19999, 3301)	1
  (19999, 3385)	1
  (19999, 3551)	1
  (19999, 3643)	1
  (19999, 3762)	1
  (19999, 3824)	2
  (19999, 3877)	2
  (19999, 3885)	1
  (19999, 3886)	2
  (19999, 3914)	2
  (19999, 3970)	2
  (19999, 4057)	1
  (19999, 4095)	1
  (19999, 4102)	1
  (19999, 4191)	1
  (19999, 4248)	1
  (19999, 4279)	1
  (19999, 4473)	1
  (19999, 4474)	1
  (19999, 4619)	1
  (19999, 4699)	1
  (19999, 4766)	1
  (19999, 4811)	1
  (19999, 4853)	1
  (19999, 4892)	1


In [18]:
from sklearn.ensemble import RandomForestClassifier

## Initialize a Random Forest classifier with 50 trees
# hyperparameter n_estimators always set in instantiation

forest = RandomForestClassifier(n_estimators = 100)

In [19]:
%%time
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the target variable

forest = forest.fit(train_bag, y_train) # can take 20 seconds to run
train_bag

Wall time: 1min 41s


<20000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1584853 stored elements in Compressed Sparse Row format>

In [20]:
# Make predictions

train_predictions = forest.predict(train_bag)
valid_predictions = forest.predict(test_bag)

In [21]:
metrics.accuracy_score(y_train,train_predictions) # 100% training accuracy

1.0

In [22]:

metrics.accuracy_score(y_test,valid_predictions) # 83% test

0.847

In [23]:

# Confusion matrix
# Is the number of False Positives and True negatives approx 50/50?
metrics.confusion_matrix(y_test,valid_predictions)

array([[2178,  370],
       [ 395, 2057]], dtype=int64)

In [24]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test,valid_predictions).ravel()
fp, fn

(370, 395)

In [45]:
df1=vectorizer.transform([review_cleaner("This was a bad movie")])


In [46]:
forest.predict(df1)

array([0], dtype=int64)

In [47]:
from sklearn.pipeline import Pipeline

In [53]:
steps = [('Vectorizer',CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None,max_features = 5000)),("Model",RandomForestClassifier(n_estimators = 100))]

In [54]:
pipeline = Pipeline(steps)

In [55]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('Vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=5000, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [56]:
y_pred= pipeline.predict(X_test)

In [64]:
pipeline.predict(["A Very Bad Movie"])

array([0], dtype=int64)

In [57]:
metrics.confusion_matrix(y_test,y_pred)

array([[2173,  375],
       [ 385, 2067]], dtype=int64)

In [61]:

# Saving model to disk
import pickle
pickle.dump(pipeline, open('Sentiment_Analysis.pkl','wb'))

In [65]:
import sqlite3

conn = sqlite3.connect('database.db')
print ("Opened database successfully")

conn.execute('CREATE TABLE Results (Review TEXT PRIMARY KEY, Sentiment TEXT)')
print ("Table created successfully")
conn.close()

Opened database successfully
Table created successfully
